In [5]:
from dotenv import load_dotenv
import os

from kaggle.api.kaggle_api_extended import KaggleApi

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Cargar el archivo .env
load_dotenv()

True

In [2]:
# Inicializa la API de Kaggle
api = KaggleApi()
api.authenticate()

# Descarga un dataset
dataset = 'stealthtechnologies/predict-student-performance-dataset'  # Cambia por el dataset que desees

dataset_folder = dataset.split(sep='/')[-1]

output_path = f'datasets/{dataset_folder}/'  # Carpeta donde se descargará
api.dataset_download_files(dataset, path=output_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/stealthtechnologies/predict-student-performance-dataset


In [3]:
df = pd.read_csv(f'{output_path}/data.csv')
df

,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,3.4,8.2,53.0,47.0
1,0.85566,3.2,5.9,55.0,35.0
2,0.68025,3.2,9.3,41.0,32.0
3,0.25936,3.2,8.2,47.0,34.0
4,0.60447,3.8,10.0,75.0,33.0
...,...,...,...,...,...
1383,0.44549,5.5,8.0,51.0,41.0
1384,0.52466,4.9,6.5,63.0,37.0
1385,0.88197,3.9,6.2,54.0,36.0
1386,0.47336,3.5,7.3,61.0,34.0


In [9]:
x = df.drop('Grades',axis=1)
y = df['Grades']

x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7,random_state=7)

rf = RandomForestRegressor()
model = rf.fit(x_train, y_train)
r2_score = model.score(x_test, y_test)

print(r2_score)


0.9781387590904583


In [23]:
results = pd.DataFrame(columns=['Dataset'])

results['Real_Grades'] = df['Grades']
results['Model_Predict'] = model.predict(x)

# Añadir la columna 'Dataset' con "Train" o "Test"
results['Dataset'] = 'Train'  # Inicializa con "Train"
results.loc[x_test.index, 'Dataset'] = 'Test'

results

,Dataset,Real_Grades,Model_Predict
0,Train,47.0,46.74
1,Train,35.0,35.34
2,Train,32.0,32.99
3,Test,34.0,34.07
4,Train,33.0,33.29
...,...,...,...
1383,Train,41.0,40.56
1384,Train,37.0,36.69
1385,Test,36.0,38.61
1386,Train,34.0,34.10


In [34]:
results['Perc_Accuracy'] = results.apply(lambda row: (row['Model_Predict']/(row['Real_Grades'])), axis=1)
results['Error'] = results.apply(lambda row: 'Subestimado' if row['Perc_Accuracy']<0.95
                                else('Preciso' if row['Perc_Accuracy']>=0.95 and row['Perc_Accuracy']<=1.05
                                    else('Sobreestimado' if row['Perc_Accuracy']>1.05 else 0)), axis=1)

results['Error_Diff'] = results.apply(lambda row: 'Subestimado' if (row['Model_Predict']-row['Real_Grades'])<-1
                                else('Preciso' if (row['Model_Predict']-row['Real_Grades'])>=-1 and (row['Model_Predict']-row['Real_Grades'])<=1
                                    else('Sobreestimado' if (row['Model_Predict']-row['Real_Grades'])>1. else 0)), axis=1)

results

,Dataset,Real_Grades,Model_Predict,Perc_Accuracy,Error,Error_Diff
0,Train,47.0,46.74,0.994468,Preciso,Preciso
1,Train,35.0,35.34,1.009714,Preciso,Preciso
2,Train,32.0,32.99,1.030938,Preciso,Preciso
3,Test,34.0,34.07,1.002059,Preciso,Preciso
4,Train,33.0,33.29,1.008788,Preciso,Preciso
...,...,...,...,...,...,...
1383,Train,41.0,40.56,0.989268,Preciso,Preciso
1384,Train,37.0,36.69,0.991622,Preciso,Preciso
1385,Test,36.0,38.61,1.072500,Sobreestimado,Sobreestimado
1386,Train,34.0,34.10,1.002941,Preciso,Preciso


In [33]:
df_resume = results[['Dataset','Error', 'Perc_Accuracy']].groupby(['Dataset','Error']).count().reset_index()
df_resume

,Dataset,Error,Perc_Accuracy
0,Test,Preciso,380
1,Test,Sobreestimado,20
2,Test,Subestimado,17
3,Train,Preciso,968
4,Train,Sobreestimado,1
5,Train,Subestimado,2


In [35]:
df_resume = results[['Dataset','Error_Diff', 'Perc_Accuracy']].groupby(['Dataset','Error_Diff']).count().reset_index()
df_resume

,Dataset,Error_Diff,Perc_Accuracy
0,Test,Preciso,247
1,Test,Sobreestimado,83
2,Test,Subestimado,87
3,Train,Preciso,922
4,Train,Sobreestimado,16
5,Train,Subestimado,33
